In [1]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
from torch.utils.data import DataLoader
import torchvision
from torchvision import datasets,models,transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()  

uinput_path = "../content/DATA_CHAMBER_2021/"
use_gpu = torch.cuda.is_available()
if use_gpu:
    print("Using CUDA")

Using CUDA


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -uq "/content/drive/My Drive/Colab Notebooks/DATA_CHAMBER_2021.zip" -d "./"

In [4]:
class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method that dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [5]:
tensor = transforms.ToTensor()
resize = transforms.Resize(256)
crop_224 = transforms.CenterCrop(224)
crop_64 = transforms.CenterCrop(64)
rotation = transforms.RandomRotation(degrees=20)
blur = transforms.GaussianBlur(kernel_size=3)
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])


def transform(type):
  if(type == "raw"):
    train_transform =  transforms.Compose([crop_224, tensor])
    test_transform = transforms.Compose([resize, crop_224, tensor])
  if(type == "scale"):
    train_transform =  transforms.Compose([resize, crop_64, tensor])
    test_transform = transforms.Compose([resize, crop_64, tensor])
  if(type == "preprocess"):
    train_transform =  transforms.Compose([resize, crop_224, blur, tensor])
    test_transform = transforms.Compose([resize, crop_224, tensor])
  if(type == "augmentation"):
    train_transform =  transforms.Compose([resize, rotation, crop_224, tensor])
    test_transform = transforms.Compose([resize, crop_224, tensor])
  return train_transform, test_transform

In [6]:
train_dataset_path = '../content/DATA_CHAMBER_2021/train'
test_dataset_path = '../content/DATA_CHAMBER_2021/test'

In [7]:
def prepare_dataset(train_transform, test_transform):
  train_datasets = ImageFolderWithPaths(root = train_dataset_path, transform = train_transform)
  test_datasets = ImageFolderWithPaths(root = test_dataset_path, transform = test_transform)
  return train_datasets, test_datasets

def prepare_dataloader(train_datasets, test_datasets):
  train_loader = DataLoader(train_datasets, batch_size = 8, shuffle = True, num_workers = 2)
  test_loader = DataLoader(test_datasets, batch_size = 8, shuffle = False, num_workers = 2)
  return train_loader, test_loader

In [8]:
def set_device():
    if torch.cuda.is_available():
        dev = "cuda:0"
    else:
        dev = "cpu"
    return torch.device(dev)
set_device()

device(type='cuda', index=0)

In [17]:
vgg16 = models.vgg19_bn(pretrained=True)
vgg19 = models.vgg16_bn(pretrained=True)

def create_model(name_model):
  if(name_model == "vgg16"):
    vgg = vgg16
  if(name_model == "vgg19"):
    vgg = vgg19
  for param in vgg.features.parameters():
    param.require_grad = False
  # Newly created modules have require_grad=True by default
  num_features = vgg.classifier[6].in_features
  features = list(vgg.classifier.children())[:-1] # Remove last layer
  features.extend([nn.Linear(num_features, 3)]) # Add our layer with 4 outputs
  vgg.classifier = nn.Sequential(*features) # Replace the model classifier
  return vgg

In [23]:
def train_model(model, train_loader, train_datasets, criterion, optimizer, num_epochs):
    train_batches = len(train_loader)
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)

        
        model.train()
        running_loss = 0.0
        running_corrects = 0
        for i,data in enumerate(train_loader):
          inputs, labels,_ = data
          print("\rTraining batch {}/{}".format(i+1, train_batches), end='', flush=True)
          inputs = inputs.to(device)
          labels = labels.to(device)
          
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()

          _, preds = torch.max(outputs, 1)
          running_loss += loss.item() * inputs.size(0)
          running_corrects += torch.sum(preds == labels.data)

          epoch_loss = running_loss / len(train_datasets)
          epoch_acc = running_corrects.double() / len(train_datasets)

        print('{} loss: {:.4f}, acc: {:.4f}'.format("train",
                                                        epoch_loss,
                                                        epoch_acc))
    return model

In [11]:
def test_model(model, test_loader, criterion):
    labels_input=list()
    labels_output=list()
    vid_id = list()
        
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    for images, labels, fname in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        labels_input= labels_input + labels.tolist()
        for f in fname:
          vid_id.append(f.split('/')[-1].split('.')[0].split('_')[0])
        outputs = model(images)
            
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
            
        labels_output= labels_output + preds.tolist()
    return labels_input,labels_output, vid_id

In [12]:
criterion = nn.CrossEntropyLoss()
device = set_device()
optimizer_ft = optim.SGD(vgg16.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [24]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
import pandas as pd
modelz = ["vgg16", "vgg19"]
data = ["raw", "scale", "preprocess", "augmentation"]
def main():
  for namemodel in modelz:
    models = create_model(name_model = namemodel)
    device = set_device()
    model = models.to(device)
    for typ in data:
      print(namemodel + " - " + typ + ":\n")
      train_transform, test_transform = transform(type = typ)
      train_datasets, test_datasets = prepare_dataset(train_transform, test_transform)
      train_loader, test_loader = prepare_dataloader(train_datasets, test_datasets)
      model = train_model(model, train_loader, train_datasets, criterion, optimizer_ft, num_epochs=5)
      y_true,y_pred,vid_id = test_model(model,test_loader, criterion)
      print(classification_report(y_true,y_pred))
      accuracy_score(y_true, y_pred)


In [ ]:
main()

vgg16 - raw:

Epoch 1/5
----------
Training batch 840/840train loss: 1.1476, acc: 0.3228
Epoch 2/5
----------
Training batch 840/840train loss: 1.1429, acc: 0.3384
Epoch 3/5
----------
Training batch 840/840train loss: 1.1472, acc: 0.3292
Epoch 4/5
----------
Training batch 840/840train loss: 1.1426, acc: 0.3356
Epoch 5/5
----------
Training batch 840/840train loss: 1.1446, acc: 0.3301
              precision    recall  f1-score   support

           0       0.21      0.08      0.12       409
           1       0.30      0.43      0.35       367
           2       0.63      0.70      0.66       831

    accuracy                           0.48      1607
   macro avg       0.38      0.40      0.38      1607
weighted avg       0.45      0.48      0.45      1607

vgg16 - scale:

Epoch 1/5
----------
Training batch 840/840train loss: 1.1848, acc: 0.3232
Epoch 2/5
----------
Training batch 840/840train loss: 1.1895, acc: 0.3245
Epoch 3/5
----------
Training batch 840/840train loss: 1.1867, a